In [ ]:
import pyproj 
import math
import requests
import os
from pathlib import Path
import rasterio
import numpy as np

## Convert productivity raster to RGB

In [ ]:
# pixel value to rgb lookup table
lookup_table = {
    1: (93,129,255),
    2: (93,192,255),
    3: (93,255,216),
    4: (93,255,109),
    5: (240,255,93),
    6: (255,171,93),
    7: (255,93,93),
}

# read the raster
with rasterio.open('sources/productivity.tif') as src:
    single_band = src.read(1)
    profile = src.profile
    
# Create empty RGB arrays
rows, cols = single_band.shape
rgb_bands = np.zeros((3, rows, cols), dtype=np.uint8)

# Map the lookup table values to RGB bands
for value, (r, g, b) in lookup_table.items():
    mask = single_band == value
    rgb_bands[0][mask] = r  # Red band
    rgb_bands[1][mask] = g  # Green band
    rgb_bands[2][mask] = b  # Blue band

# Update metadata for a 3-band output
profile.update(
    dtype=rasterio.uint8,
    count=3,
    photometric='RGB'
)

# Write the RGB raster
with rasterio.open("sources/productivity_rgb.tif", "w", **profile) as dst:
    dst.write(rgb_bands)

## Convert PNV subzones to zones

In [ ]:
subzone_to_zone_lookup = {
    46: 9, # sitka spruce wet 
    47: 9, # sitka spruce very wet
    48: 17, # redwood very moist
    49: 17, # redwood wet 
    91: 23, # mountain hemlock cold very moist
    92: 23, # mountain hemlock cool very moist
    93: 23, # mountain hemlock cool wet
    94: 23, # mountain hemlock cool very wet
    87: 22, # silver fir mild wet
    88: 22, # silver fir mild very wet
    89: 22, # silver fir cold wet
    82: 19, # western hemlock moist
    83: 19, # western hemlock very moist
    84: 19, # western hemlock wet
    76: 18, # western redcedar mild
    77: 18, # western redcedar cool
    61: 25, # subalpine fir - englemann spruce cold moist
    62: 25, # subalpine fir - englemann spruce cool dry
    63: 25, # subalpine fir - englemann spruce cool wet
    64: 25, # subalpine fir - englemann spruce cool very moist
    56: 20, # white fir - grand fir warm xeric 
    57: 20, # white fir - grand fir warm dry
    58: 20, # white fir - grand fir warm moist
    59: 20, # white fir - grand fir cool moist
    71: 21, # california red fir - shasta red fir cool moist
    72: 21, # california red fir - shasta red fir mild very moist
    73: 21, # california red fir - shasta red fir cool very moist 
    79: 13, # port orford cedar
    51: 15, # tanoak very moist
    52: 15, # tanoak very wet
    40: 14, # Douglas-fir bigcone douglas fir 
    41: 14, # Douglas-fir very warm dry
    42: 14, # Douglas-fir warm dry 
    43: 14, # Douglas-fir very warm moist
    37: 12, # jeffrey pine warm xeric 
    38: 12, # jeffrey pine warm dry 
    39: 12, # jeffrey pine knobcone pine 
    30: 10, # ponderosa pine - juniper woodlands 
    31: 10, # ponderosa pine warm xeric 
    33: 10, # ponderosa pine - lodgepole pine 
    34: 10, # ponderosa pine - oak
    -114: 30, # parklands subalpine grassland-forbland 
    -113: 30, # parklands subalpine shrub
    65: 30, # parklands subalpine fir parklands 
    95: 30, # parklands mountain hemlock parklands 
    96: 30, # parklands limber pine 
    97: 30, # parklands bristlecone-foxtail pine 
    98: 30, # parklands western white pine 
    99: 30, # parklands whitebark pine parklands 
    100: 30, # parklands subalpline larch parklands 
    101: 30, # parklands sierra lodgepole pine parklands 
    25: 8, # lodgepole pine cool dry 
    27: 8, # lodgepole pine wetlands 
    29: 8, # lodgepole pine monterey pine - bishop pine 
    35: 7, # california coulter pine - oak 
    36: 7, # california foothill pine - oak 
    21: 6, # juniper scablands 
    22: 6, # juniper woodlands
    23: 6, # pinyon woodlands
    24: 6, # cypress woodlands
    15: 11, # riparian shrub 
    16: 11, # riparian hardwood forest
    17: 11, # other hardwoods
    18: 11, # oak woodlands
    19: 11, # desert hardwoods
    -9: 4, # wet meadow
    -8: 4, # moist meadow
    -7: 4, # dry meadow
    -6: 4, # upload grass
    -5: 4, # scabland grass
    -17: 5, # creosote - bursage scrub
    -16: 5, # desert scrub
    -15: 5, # coastal scrub
    -14: 5, # chaparral
    -13: 5, # montane shrub
    -12: 5, # upland shrub
    -11: 5, # scabland shrub
    -10: 5, # salt desert shrub
}

In [ ]:

# Read the input raster
with rasterio.open('sources/pnv.tif') as src:
    pnv_data = src.read(1)
    profile = src.profile

# Create output array with same shape as input
output = np.zeros_like(pnv_data)

# Replace values using the subzone_to_zone_lookup
for subzone, zone in subzone_to_zone_lookup.items():
    output[pnv_data == subzone] = zone

# Write the output raster
with rasterio.open('sources/pnv_zones.tif', 'w', **profile) as dst:
    dst.write(output, 1)

## Convert PNV zones raster to RGB

In [ ]:
zone_to_hex_lookup = {
    # reds 
    9: "#dd868c", # sitka spruce
    17: "#d73d46", # redwood
    # greens 
    23: "#3c6430", # mountain hemlock
    19: "#5a9d46", # western hemlock
    14: "#61cf55", # douglas-fir
    # blue - purple
    22: "#53b1d1", # silver fir
    25: "#928bd4", # subalpine fir - englemann spruce
    20: "#523e80", # white fir - grand fir
    21: "#7c48c6", # california red fir - shasta red fir
    # pinks
    13: "#d555b2", # port orford cedar
    15: "#91355f", # tanoak
    # orange
    12: "#863d2a", # jeffrey pine
    10: "#d46f31", # ponderosa pine
    # yellows
    30: "#d4ac3f", # parklands 
    8: "#84712f", # lodgepole pine
    7: "#b5ba79", # california coulter pine - oak
    # browns
    6: "#785a0a", # juniper
    # yellows
    11: "#ddcc77", # riparian / oak woodlands
    4: "#74ab70", # meadow
    5: "#bf9841", # scrub
}

In [ ]:
from matplotlib.colors import to_rgb

# Convert hex colors to RGB tuples (0-255 range)
rgb_lookup = {}

for zone, hex_code in zone_to_hex_lookup.items():
    (r,g,b) = to_rgb(hex_code)
    rgb_lookup[zone] = (
        1 if r * 255 == 0 else r * 255,
        1 if g * 255 == 0 else g * 255,
        1 if b * 255 == 0 else b * 255
    )
    print(rgb_lookup[zone])

# Read the input raster
with rasterio.open('sources/pnv_zones.tif') as src:
    zone_data = src.read(1)
    profile = src.profile

# Create empty RGB arrays 
rows, cols = zone_data.shape
rgb_bands = np.zeros((3, rows, cols), dtype=np.uint8)

# Map the zone colors to RGB bands
for zone, (r, g, b) in rgb_lookup.items():
    mask = zone_data == zone
    rgb_bands[0][mask] = r  # Red band
    rgb_bands[1][mask] = g  # Green band
    rgb_bands[2][mask] = b  # Blue band

# Update profile for RGB output
profile.update(
    dtype=rasterio.uint8,
    count=3,
    photometric='RGB'
)

# Write the RGB raster
with rasterio.open('sources/pnv_zones_rgb.tif', 'w', **profile) as dst:
    dst.write(rgb_bands)


## Convert landfire zones to match PNV zones

Find vegetation classes here: https://www.landfire.gov/vegetation/evt

In [ ]:
landfire_to_zone_lookup = {
    7008: 11,
    7010: 25,
    7015: 17,
    7017: 6,
    7018: 20,
    7019: 6,
    7021: 13, 
    7022: 13, 
    7027: 14,
    7028: 20,
    7029: 11, 
    7030: 11, 
    7031: 12, 
    7032: 21,
    7033: 30, 
    7035: 14,
    7036: 9,
    7037: 19,
    7038: 30, 
    7039: 19,
    7041: 23,
    7042: 22,
    7043: 15,
    7044: 23,
    7045: 14,
    7046: 30,
    7047: 20, 
    7050: 8,
    7053: 10,
    7055: 25, 
    7056: 25,
    7057: 30,
    7058: 8,
    7060: 10,
    7061: 11, 
    7062: 11, 
    7063: 11,
    7065: 0,
    7068: 30,
    7069: 0,
    7084: 5,
    7096: 5,
    7098: 5,
    7105: 5,
    7128: 5,
    7156: 11,
    7158: 11,
    7167: 8,
    7172: 20,
    7174: 22,
    7177: 15, 
    7260: 7,
    7261: 7,
    7262: 11,
    7263: 10,
}

In [ ]:
# Read the input raster
with rasterio.open('sources/landfire.tif') as src:
    landfire_data = src.read(1)
    profile = src.profile

# Create output array with same shape as input
output = np.zeros_like(landfire_data)

# Replace values using the landfire_to_zone_lookup
for landfire_val, zone in landfire_to_zone_lookup.items():
    output[landfire_data == landfire_val] = zone

# Write the output raster
with rasterio.open('sources/landfire_zones.tif', 'w', **profile) as dst:
    dst.write(output, 1)
